In [48]:
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import peakutils
from peakutils.plot import plot as pplot
from matplotlib import pyplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

import talib

In [49]:
def parse_trades(filepath):
    trades = pd.read_csv(filepath, header=0,
                         parse_dates=True, index_col=0)
    trades['open_date'] = pd.to_datetime(trades['open_date'])
    trades['close_date'] = pd.to_datetime(trades['close_date'])

    return trades

class CSVDataHandler:
    def __init__(self, csv_dir, symbol_list, timeframe):
        self.csv_dir = csv_dir
        self.symbol_list = symbol_list
        self.timeframe = timeframe
        
        self.symbol_data = dict()
        self.latest_symbol_data = dict()
        
        self.continue_backtest = True
        
        self._import_symbol_data()
        
    def _import_symbol_data(self):
        csv_files_path = f'{Path().absolute()}/{self.csv_dir}'
        columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        combined_symbol_index = None
        
        for symbol in symbol_list:
            symbol_csv_path = f'{csv_files_path}/{symbol}_{self.timeframe}.csv'
            self.symbol_data[symbol] = pd.read_csv(symbol_csv_path, header=None, index_col=0, names=columns)
            
            self.symbol_data[symbol].index = pd.to_datetime(self.symbol_data[symbol].index, unit='ms')
            self.symbol_data[symbol] = self.symbol_data[symbol].drop_duplicates()
            self.symbol_data[symbol] = self.symbol_data[symbol].sort_index()
            
            if combined_symbol_index is None:
                combined_symbol_index = self.symbol_data[symbol].index
            else:
                combined_symbol_index = combined_symbol_index.union(self.symbol_data[symbol].index)
            
            self.latest_symbol_data[symbol] = list()
            
        for symbol in symbol_list:
            self.symbol_data[symbol] = self.symbol_data[symbol].reindex(index=combined_symbol_index, method='pad', fill_value=0).itertuples(name='OHLCV')
            
    def _get_new_bar(self, symbol):
        for bar in self.symbol_data[symbol]:
            yield bar
            
    def update_bars(self):
        for symbol in self.symbol_list:
            try:
                bar = next(self._get_new_bar(symbol))
            except StopIteration:
                self.continue_backtest = False
            else:
                if bar is not None:
                    self.latest_symbol_data[symbol].append(bar)
                    
    def get_latest_bar(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars(self, symbol, N=1):
        try:
            return np.array(self.latest_symbol_data[symbol][-N:])
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_datetime(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1].Index
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_value(self, symbol, value_type):
        try:
            return getattr(self.latest_symbol_data[symbol][-1], value_type)
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars_values(self, symbol, value_type, N=1):
        try:
            bars = self.latest_symbol_data[symbol][-N:]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
        else:
            return np.array([getattr(bar, value_type) for bar in bars])

In [50]:
symbol_list = ['BTC-USDT']
data_handler = CSVDataHandler('../exchange_data', symbol_list, timeframe='1h')

In [52]:
for _ in range(250):
    data_handler.update_bars()

data = data_handler.latest_symbol_data['BTC-USDT']
data_handler.get_latest_bars('BTC-USDT', N=3)

array([[Timestamp('2017-09-07 04:00:00'), 4489.48, 4549.52, 4482.0,
        4541.17, 18.430628],
       [Timestamp('2017-09-07 05:00:00'), 4542.35, 4561.9, 4511.97,
        4519.48, 20.847166],
       [Timestamp('2017-09-07 06:00:00'), 4519.47, 4548.13, 4498.39,
        4498.4, 11.892871000000001]], dtype=object)

In [5]:
rsi = talib.RSI(close, timeperiod=20)
trades_path = f'../backtest_results/trades.csv'
trades = parse_trades(trades_path)

In [12]:
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=
           [[{"type": "scatter"}],
           [{"type": "scatter"}]]
)

    data['close'].resample('D').agg({
        'close': lambda x: x[-1]
    })

In [1]:
fig.add_trace(
    go.Scatter(
        x=datetimes,
        y=close,
        mode="lines",
        name="Closes",
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=trades['open_date'],
        y=trades['open_market_price'],
        mode="markers",
        marker=dict(symbol="triangle-up", size=10),
        marker_color='rgba(8, 166, 53, 1)',
        name="Open position"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=trades['close_date'],
        y=trades['close_market_price'],
        mode="markers",
        marker=dict(symbol="triangle-down", size=10),
        marker_color='rgba(219, 22, 22, 1)',
        name="Close position"
    ),
    row=1, col=1
)


fig.add_trace(
    go.Scatter(
        x=datetimes,
        y=rsi,
        mode="lines",
        name="RSI"
    ),
    row=2, col=1
)


fig.update_layout(
    height=800,
    showlegend=False,
    title_text="BTC/USDT",
)

fig.show()

NameError: name 'fig' is not defined